# Model for Role Requirements

## Import Libraries

In [288]:
import pandas
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import numpy as np



## Get Profile from an Employee

In [289]:
# Get employee's profile

# List skills that the person has and also the list of interests
person_skills = ['Hadoop','Apache Spark']
person_interests = ['Apache Spark', 'Hadoop']
level_requirement = 'Architect'#'Mid-level' # not used in this match

# All skills available
skills_df = pandas.read_csv(filepath_or_buffer="data/skill-list.csv", sep=",", encoding="latin1")
skills_df

person_all_skills = person_skills + person_interests
#person_interests = []
#for item in person_all_skills:
#    person_interests.append(item+"-Want")


## Transform Role Requirements Dataset

In [290]:
# Read source dataset
df = pandas.read_csv(filepath_or_buffer="data/skill2vec_50k.csv", sep=",", encoding="latin1", header = None)
df = df.fillna('')
df

/Users/vanessamueller/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,

,0,1,2,3,4,5,6,7,8,9,...,951,952,953,954,955,956,957,958,959,960
0,125720,HR Executive,screening,selection,Interview,HR,Recruiter,IT Recruiter,Sourcing,recruitment executive,...,,,,,,,,,,
1,112708,Special Teacher,Teaching,Education,,,,,,,...,,,,,,,,,,
2,115226,consulting,fresher,IT helpdesk,Techincal Troubleshooting,international voice,international BPO,technical support,outsourcing,call center,...,,,,,,,,,,
3,19805,diploma,machining,cnc m,mould,conventional machines,die making,knowledge,tool,cipet,...,,,,,,,,,,
4,80208,Compensation,Benefits,HR Functions,Alm,Payroll,ESS,Core HR,QC,QA,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,27374,Chief Engineer,,,,,,,,,...,,,,,,,,,,
49996,88457,Receptionist Activities,Front Desk,front office,front desk executive,front office executive,receptionist,reception,,,...,,,,,,,,,,
49997,34590,SQL Queries,Log Analysis,Hardware Networking,People Leadership,Technical Skills,Unix,Oracle,Service Delivery Management,Database Administration,...,,,,,,,,,,
49998,86171,Quality Analyst,,,,,,,,,...,,,,,,,,,,


In [291]:
# Join all skills in a same column
#df = df.sample(1000)

df["Skills"] = df[1]
for x in range(1,961):
    try:
        df["Skills"] = df["Skills"] + ";" + df[x]
    except:
        print(df[x])

df["id"] = df[0]
df = df[["id", "Skills"]]
df


0         
1         
2         
3         
4         
        ..
49995     
49996     
49997     
49998     
49999     
Name: 96, Length: 50000, dtype: object
0         
1         
2         
3         
4         
        ..
49995     
49996     
49997     
49998     
49999     
Name: 461, Length: 50000, dtype: object
0         
1         
2         
3         
4         
        ..
49995     
49996     
49997     
49998     
49999     
Name: 462, Length: 50000, dtype: object
0         
1         
2         
3         
4         
        ..
49995     
49996     
49997     
49998     
49999     
Name: 659, Length: 50000, dtype: object
0         
1         
2         
3         
4         
        ..
49995     
49996     
49997     
49998     
49999     
Name: 660, Length: 50000, dtype: object


,id,Skills
0,125720,HR Executive;HR Executive;screening;selection;...
1,112708,Special Teacher;Special Teacher;Teaching;Educa...
2,115226,consulting;consulting;fresher;IT helpdesk;Tech...
3,19805,diploma;diploma;machining;cnc m;mould;conventi...
4,80208,Compensation;Compensation;Benefits;HR Function...
...,...,...
49995,27374,Chief Engineer;Chief Engineer;;;;;;;;;;;;;;;;;...
49996,88457,Receptionist Activities;Receptionist Activitie...
49997,34590,SQL Queries;SQL Queries;Log Analysis;Hardware ...
49998,86171,Quality Analyst;Quality Analyst;;;;;;;;;;;;;;;...


In [292]:
# Account for synonyms


In [293]:
# Get list of skills - Source: Stack Overflow Survey questions
#skills_df = pandas.read_csv(filepath_or_buffer="data/skill-list.csv", sep=",", encoding="latin1")
#skills_df

# Get only positions with skills that are mapped
#skills = skills_df['skill']

# Get only skills that are relevant for the person
skills = person_all_skills

# Account for synonyms
synonym_skill = skills_df['synonym']
def find_index(arr, val):
    for i in range(len(arr)):
        if arr[i] == val:
            return i
    return -1

for skill in skills:
    df[skill] = df["Skills"].apply(lambda x: 1 if skill.lower() in [item.lower() for item in x.split(';')] or synonym_skill[find_index(skills, skill)] in [item.lower() for item in x.split(';')] else 0)
    #df[skill] = df["Skills"].apply(lambda x: 1 if skill in x.split(';') or synonym_skill[find_index(skills, skill)] in x.split(';') else 0)
df


<ipython-input-293-ef0f1d3b8394>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[skill] = df["Skills"].apply(lambda x: 1 if skill.lower() in [item.lower() for item in x.split(';')] or synonym_skill[find_index(skills, skill)] in [item.lower() for item in x.split(';')] else 0)


,id,Skills,Hadoop,Apache Spark
0,125720,HR Executive;HR Executive;screening;selection;...,0,0
1,112708,Special Teacher;Special Teacher;Teaching;Educa...,0,0
2,115226,consulting;consulting;fresher;IT helpdesk;Tech...,0,0
3,19805,diploma;diploma;machining;cnc m;mould;conventi...,0,0
4,80208,Compensation;Compensation;Benefits;HR Function...,0,0
...,...,...,...,...
49995,27374,Chief Engineer;Chief Engineer;;;;;;;;;;;;;;;;;...,0,0
49996,88457,Receptionist Activities;Receptionist Activitie...,0,0
49997,34590,SQL Queries;SQL Queries;Log Analysis;Hardware ...,0,0
49998,86171,Quality Analyst;Quality Analyst;;;;;;;;;;;;;;;...,0,0


In [294]:
#synonym_skill[skills.index("Python")]
#skills[find_index(skills, "Microsoft Azure")].lower()
#skills

# Remove unused columns and replicate skills columns to interests
for skill in skills:
    df[skill+'-Want'] = df[skill]
df

df = df.drop(columns=["Skills"])
df

<ipython-input-294-70624de7293d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[skill+'-Want'] = df[skill]


,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want
0,125720,0,0,0,0
1,112708,0,0,0,0
2,115226,0,0,0,0
3,19805,0,0,0,0
4,80208,0,0,0,0
...,...,...,...,...,...
49995,27374,0,0,0,0
49996,88457,0,0,0,0
49997,34590,0,0,0,0
49998,86171,0,0,0,0


In [295]:
# Count how many matched skills there are
df["CountSkills"] = 0
for skill in skills:
    df["CountSkills"] = df["CountSkills"] + df[skill]
#df

# Get only requirements that have relevant skills
df = df.loc[df['CountSkills'] > 0]
df.sort_values(by="CountSkills", ascending=0)


,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want,CountSkills
39087,35141,1,1,1,1,4
6830,96798,1,1,1,1,4
33726,21848,1,1,1,1,4
38672,21943,1,1,1,1,4
31656,36161,1,0,1,0,2
...,...,...,...,...,...,...
18376,49129,1,0,1,0,2
18391,106357,1,0,1,0,2
18403,9500,1,0,1,0,2
18431,63124,1,0,1,0,2


In [296]:
# Remove columns where it's always 0
#df = df.loc[:, (df != 0).any(axis=0)]
#df
df = df.drop(columns=["CountSkills"])
df

,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want
87,33174,1,0,1,0
350,127459,1,0,1,0
703,21974,1,0,1,0
780,21955,1,0,1,0
976,7865,1,0,1,0
...,...,...,...,...,...
49110,35394,1,0,1,0
49280,108684,1,0,1,0
49419,67326,1,0,1,0
49912,114110,1,0,1,0


In [297]:
# Verify if any rows for a particular skill
#df.loc[df['Microsoft Azure'] == 1].loc[df['Python'] == 1]
#df.loc[df['Microsoft Azure'] == 1].loc[df['Python'] == 1].loc[df['Kubernetes'] == 1]
#df.loc[df['Python'] == 1].loc[df['Java'] == 1].loc[df['Docker'] == 1]

#df.loc[df['Python'] == 1]

#df.loc[df['Microsoft SQL Server'] == 1]





In [298]:
# Create an array for the person
size = len(df.drop(columns='id').columns)
#size
person = np.zeros(shape=(1, size))

# Add code to have 1 when skill and or when interest
count = 0
for skill in list(df.drop(columns='id').columns):
    #print(skill)
    
    if ("Want" in skill):
        skill = skill.replace("-Want", "")
        person[0][count] = 1 if skill in person_interests else 0
        print(skill, " - ", person[0][count])
    else:
        person[0][count] = 1 if skill in person_skills else 0
        print(skill, " - ", person[0][count])
    count = count +1

person



Hadoop  -  1.0
Apache Spark  -  1.0
Hadoop  -  1.0
Apache Spark  -  1.0


array([[1., 1., 1., 1.]])

## KMeans to clusterize Roles

In [299]:
# Group people into clusters
number_clusters = df.drop(columns=["id"]).drop_duplicates().count()[0]
number_clusters
model = KMeans(n_clusters=number_clusters, random_state=42).fit(df.drop(columns=["id"]))
clusters = model.labels_
center = model.cluster_centers_

df
df_clusters = df
df_clusters["cluster"] = clusters
df_clusters

,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want,cluster
87,33174,1,0,1,0,0
350,127459,1,0,1,0,0
703,21974,1,0,1,0,0
780,21955,1,0,1,0,0
976,7865,1,0,1,0,0
...,...,...,...,...,...,...
49110,35394,1,0,1,0,0
49280,108684,1,0,1,0,0
49419,67326,1,0,1,0,0
49912,114110,1,0,1,0,0


In [300]:
#print(df_clusters.drop(columns=["ResponseId","YearsCodePro","Skills-Have","Skills-Want"]).drop_duplicates().count())
#print(df_clusters['cluster'].max())

roles_df = df_clusters.sort_values("cluster")
roles_df


,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want,cluster
87,33174,1,0,1,0,0
30426,98064,1,0,1,0,0
30721,49063,1,0,1,0,0
30963,65168,1,0,1,0,0
31085,21853,1,0,1,0,0
...,...,...,...,...,...,...
17466,21859,1,0,1,0,0
33726,21848,1,1,1,1,1
39087,35141,1,1,1,1,1
6830,96798,1,1,1,1,1


## KNN to find the Best Matches

In [301]:
# Find the one employee cluster closer to the requirement
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(roles_df.drop(columns=["id", "cluster"]), roles_df['cluster'])

prediction = knn.predict(person)

prediction

array([1], dtype=int32)

In [302]:
#df
person

array([[1., 1., 1., 1.]])

## Get list of the matched Employees

In [303]:
# Employee found in the match
roles_df.loc[roles_df['cluster'] == prediction[0]]
#roles_df.loc[roles_df['id'] == 9271]



,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want,cluster
33726,21848,1,1,1,1,1
39087,35141,1,1,1,1,1
6830,96798,1,1,1,1,1
38672,21943,1,1,1,1,1


In [304]:
# Confirm if there is any perfect match
#roles_df.loc[roles_df['Python'] == 1].loc[employees_df['Microsoft SQL Server'] == 1].loc[employees_df['Microsoft Azure'] == 1].loc[employees_df['Apache Spark'] == 1].loc[employees_df['Scikit-learn'] == 1].loc[employees_df['TensorFlow'] == 1].loc[employees_df['Docker'] == 1]
roles_df.loc[roles_df['Apache Spark'] == 1].loc[roles_df['Hadoop'] == 1]


,id,Hadoop,Apache Spark,Hadoop-Want,Apache Spark-Want,cluster
33726,21848,1,1,1,1,1
39087,35141,1,1,1,1,1
6830,96798,1,1,1,1,1
38672,21943,1,1,1,1,1


In [66]:
# Check information for each employee considered
import pandas
df_original = pandas.read_csv(filepath_or_buffer="data/survey_results_public_1.csv", sep=",", encoding="latin1")
df_original = df_original[["ResponseId","YearsCodePro","LanguageHaveWorkedWith","LanguageWantToWorkWith","DatabaseHaveWorkedWith","DatabaseWantToWorkWith","PlatformHaveWorkedWith","PlatformWantToWorkWith","WebframeHaveWorkedWith","WebframeWantToWorkWith","MiscTechHaveWorkedWith","MiscTechWantToWorkWith","ToolsTechHaveWorkedWith","ToolsTechWantToWorkWith"]]

df_original.loc[df_original['ResponseId'] == 31107]


,ResponseId,YearsCodePro,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith
31106,31107,3,Assembly;Bash/Shell;C;Haskell;HTML/CSS;Java;Ja...,HTML/CSS;JavaScript;Python;Scala;SQL,Microsoft SQL Server;MongoDB;MySQL;PostgreSQL;...,MySQL;PostgreSQL;SQLite,AWS;Firebase;Microsoft Azure;OVH,AWS;Firebase;Google Cloud;Microsoft Azure;OVH,Angular;Django;jQuery;Node.js;React.js,Django,Apache Spark;Hadoop;Keras;NumPy;Pandas;Scikit-...,Apache Spark;Hadoop;Keras;NumPy;Pandas;Scikit-...,Docker,Docker;Kubernetes;Terraform
